In [29]:
import re
import pandas as pd
import numpy as np
import datetime

In [2]:
%store -r mainDf
teamGameData = pd.read_csv('C:\\Users\\Michael\\Desktop\\Springboard\\Capstone2\\csvFiles\\teamGameData.csv',index_col=0)

In [3]:
# Let's first work with the odds data
mainDf.head()

,date,favorite,spread,underdog,total,awayMoneyLine,homeMoneyLine,season,week
0,"September 8, 2011 12:00 AM",at Packers,-4.0,Saints,47.5,NaN,NaN,2011,1
1,"September 11, 2011 12:00 AM",at Texans,-8.5,Colts,43.5,NaN,NaN,2011,1
2,"September 11, 2011 12:00 AM",Titans,-1.0,at Jaguars,37.5,NaN,NaN,2011,1
3,"September 11, 2011 12:00 AM",at Chiefs,-4.0,Bills,40.0,NaN,NaN,2011,1
4,"September 11, 2011 12:00 AM",at Chargers,-9.0,Vikings,42.5,NaN,NaN,2011,1


In [4]:
# Split table by favorite and underdog
favorite = mainDf[['date','season','week','favorite','spread','total','awayMoneyLine','homeMoneyLine']]
favorite.head()

,date,season,week,favorite,spread,total,awayMoneyLine,homeMoneyLine
0,"September 8, 2011 12:00 AM",2011,1,at Packers,-4.0,47.5,NaN,NaN
1,"September 11, 2011 12:00 AM",2011,1,at Texans,-8.5,43.5,NaN,NaN
2,"September 11, 2011 12:00 AM",2011,1,Titans,-1.0,37.5,NaN,NaN
3,"September 11, 2011 12:00 AM",2011,1,at Chiefs,-4.0,40.0,NaN,NaN
4,"September 11, 2011 12:00 AM",2011,1,at Chargers,-9.0,42.5,NaN,NaN


In [5]:
# create column that signifies whether favorite is home or away
favorite.loc[favorite['favorite'].str.contains('at '), 'homeOrAway'] = 'home'
favorite.loc[np.logical_not(favorite['favorite'].str.contains('at ')), 'homeOrAway'] = 'away'
favorite.tail()

,date,season,week,favorite,spread,total,awayMoneyLine,homeMoneyLine,homeOrAway
11,"December 30, 2018 4:25 PM",2018,17,at Seahawks,-15.0,40.0,675.0,-860.0,home
12,"December 30, 2018 4:25 PM",2018,17,at Chiefs,-14.0,53.0,700.0,-900.0,home
13,"December 30, 2018 4:25 PM",2018,17,at Rams,-10.5,50.0,390.0,-440.0,home
14,"December 30, 2018 4:25 PM",2018,17,at Vikings,-5.5,39.5,215.0,-235.0,home
15,"December 30, 2018 8:20 PM",2018,17,Colts,-5.5,42.5,-235.0,215.0,away


In [6]:
# Create a moneyLine column that uses awayMoneyLine if homeOrAway is away,
# and uses homeMoneyLine otherwise

awayFav = favorite.loc[favorite['homeOrAway'] == 'away']
awayFav = awayFav.drop(columns=['homeMoneyLine'])
awayFav = awayFav.rename(columns={'awayMoneyLine':'moneyLine'})

homeFav = favorite.loc[favorite['homeOrAway'] == 'home']
homeFav = homeFav.drop(columns=['awayMoneyLine'])
homeFav = homeFav.rename(columns={'homeMoneyLine':'moneyLine'})

# Reconnect the two tables
favorite = homeFav.append(awayFav)
favorite.tail()

# Create impliedTotal column
favorite['impliedTotal'] = (favorite['total']/2) - (favorite['spread']/2)
favorite.head()

,date,season,week,favorite,spread,total,moneyLine,homeOrAway,impliedTotal
0,"September 8, 2011 12:00 AM",2011,1,at Packers,-4.0,47.5,NaN,home,25.75
1,"September 11, 2011 12:00 AM",2011,1,at Texans,-8.5,43.5,NaN,home,26.00
3,"September 11, 2011 12:00 AM",2011,1,at Chiefs,-4.0,40.0,NaN,home,22.00
4,"September 11, 2011 12:00 AM",2011,1,at Chargers,-9.0,42.5,NaN,home,25.75
6,"September 11, 2011 12:00 AM",2011,1,at Cardinals,-7.0,37.5,NaN,home,22.25


In [7]:
# Now let's do the same thing for underdogs
underdog = mainDf[['date','season','week','underdog','spread','total','awayMoneyLine','homeMoneyLine']]
underdog.head()

,date,season,week,underdog,spread,total,awayMoneyLine,homeMoneyLine
0,"September 8, 2011 12:00 AM",2011,1,Saints,-4.0,47.5,NaN,NaN
1,"September 11, 2011 12:00 AM",2011,1,Colts,-8.5,43.5,NaN,NaN
2,"September 11, 2011 12:00 AM",2011,1,at Jaguars,-1.0,37.5,NaN,NaN
3,"September 11, 2011 12:00 AM",2011,1,Bills,-4.0,40.0,NaN,NaN
4,"September 11, 2011 12:00 AM",2011,1,Vikings,-9.0,42.5,NaN,NaN


In [8]:
# create column that signifies whether underdog is home or away
underdog.loc[underdog['underdog'].str.contains('at '), 'homeOrAway'] = 'home'
underdog.loc[np.logical_not(underdog['underdog'].str.contains('at ')), 'homeOrAway'] = 'away'
underdog.tail()

,date,season,week,underdog,spread,total,awayMoneyLine,homeMoneyLine,homeOrAway
11,"December 30, 2018 4:25 PM",2018,17,Cardinals,-15.0,40.0,675.0,-860.0,away
12,"December 30, 2018 4:25 PM",2018,17,Raiders,-14.0,53.0,700.0,-900.0,away
13,"December 30, 2018 4:25 PM",2018,17,49ers,-10.5,50.0,390.0,-440.0,away
14,"December 30, 2018 4:25 PM",2018,17,Bears,-5.5,39.5,215.0,-235.0,away
15,"December 30, 2018 8:20 PM",2018,17,at Titans,-5.5,42.5,-235.0,215.0,home


In [9]:
# Create a moneyLine column that uses awayMoneyLine if homeOrAway is away,
# and uses homeMoneyLine otherwise

awayDog = underdog.loc[underdog['homeOrAway'] == 'away']
awayDog = awayDog.drop(columns=['homeMoneyLine'])
awayDog = awayDog.rename(columns={'awayMoneyLine':'moneyLine'})

homeDog = underdog.loc[underdog['homeOrAway'] == 'home']
homeDog = homeDog.drop(columns=['awayMoneyLine'])
homeDog = homeDog.rename(columns={'homeMoneyLine':'moneyLine'})

# Reconnect the two tables
underdog = homeDog.append(awayDog)
underdog.tail()

# use absolute value of spread, since underdog
underdog['spread'] = underdog['spread'].abs()

# Create impliedTotal column (adjust for underdog)
underdog['impliedTotal'] = (underdog['total']/2) - (underdog['spread']/2)
underdog.tail()

,date,season,week,underdog,spread,total,moneyLine,homeOrAway,impliedTotal
10,"December 30, 2018 4:25 PM",2018,17,Bengals,13.5,46.0,575.0,away,16.25
11,"December 30, 2018 4:25 PM",2018,17,Cardinals,15.0,40.0,675.0,away,12.50
12,"December 30, 2018 4:25 PM",2018,17,Raiders,14.0,53.0,700.0,away,19.50
13,"December 30, 2018 4:25 PM",2018,17,49ers,10.5,50.0,390.0,away,19.75
14,"December 30, 2018 4:25 PM",2018,17,Bears,5.5,39.5,215.0,away,17.00


In [10]:
# now change favorite and underdog team columns to 'team', then put table back together
favorite = favorite.rename(columns={'favorite':'team'})
underdog = underdog.rename(columns={'underdog':'team'})

oddsTable = favorite.append(underdog)
oddsTable.head()

,date,season,week,team,spread,total,moneyLine,homeOrAway,impliedTotal
0,"September 8, 2011 12:00 AM",2011,1,at Packers,-4.0,47.5,NaN,home,25.75
1,"September 11, 2011 12:00 AM",2011,1,at Texans,-8.5,43.5,NaN,home,26.00
3,"September 11, 2011 12:00 AM",2011,1,at Chiefs,-4.0,40.0,NaN,home,22.00
4,"September 11, 2011 12:00 AM",2011,1,at Chargers,-9.0,42.5,NaN,home,25.75
6,"September 11, 2011 12:00 AM",2011,1,at Cardinals,-7.0,37.5,NaN,home,22.25


In [11]:
# Now eliminate 'at ' in the team column
oddsTable['team'] =  oddsTable.loc[:, 'team'].replace(regex=True, to_replace="at ", value="")
oddsTable.head()

,date,season,week,team,spread,total,moneyLine,homeOrAway,impliedTotal
0,"September 8, 2011 12:00 AM",2011,1,Packers,-4.0,47.5,NaN,home,25.75
1,"September 11, 2011 12:00 AM",2011,1,Texans,-8.5,43.5,NaN,home,26.00
3,"September 11, 2011 12:00 AM",2011,1,Chiefs,-4.0,40.0,NaN,home,22.00
4,"September 11, 2011 12:00 AM",2011,1,Chargers,-9.0,42.5,NaN,home,25.75
6,"September 11, 2011 12:00 AM",2011,1,Cardinals,-7.0,37.5,NaN,home,22.25


In [12]:
# Turn season and weeks into ints
oddsTable[['week','season']] = oddsTable[['week','season']].astype('int')

In [33]:
oddsTable.to_csv('C://Users//Michael//Desktop//Springboard//Capstone2//csvFiles//oddsTable.csv')

In [14]:
# Now let's work with the teamGameData table
teamGameData.head()

,gameId,date,team,totExp,oTot,oPass,oRush,oTov,dTot,dPass,...,cmpAttYdsTdsInts,sackedYds,netPyds,totYds,fumLost,to,penYds,3dConv,4dConv,timePoss
0,201009090nor,"Thursday Sep 9, 2010",Vikings,-5.0,-6.45,-5.54,-0.91,-4.17,-2.86,-4.60,...,15-27-171-1-1,1-9,162,253,1-0,1,6-60,5-13,0-0,26:17
1,201009090nor,"Thursday Sep 9, 2010",Saints,5.0,2.86,4.60,-1.42,0.00,6.45,5.54,...,27-36-237-1-0,1-8,229,308,1-0,0,3-20,3-11,0-0,33:43
0,201009120pit,"Sunday Sep 12, 2010",Falcons,-6.0,-8.90,-4.99,-3.91,-3.92,3.01,0.92,...,27-44-252-0-1,2-15,237,295,0-0,1,3-24,6-16,0-0,30:29
1,201009120pit,"Sunday Sep 12, 2010",Steelers,6.0,-3.01,-0.92,-1.43,-3.79,8.90,4.99,...,18-26-236-0-1,3-25,211,354,0-0,1,4-25,4-14,0-0,32:06
0,201009120oti,"Sunday Sep 12, 2010",Titans,25.0,10.04,6.91,3.79,-4.66,14.46,12.28,...,22-37-180-1-1,4-30,150,286,4-1,2,10-77,3-14,0-2,28:56


In [15]:
teamGameData.head()

,gameId,date,team,totExp,oTot,oPass,oRush,oTov,dTot,dPass,...,cmpAttYdsTdsInts,sackedYds,netPyds,totYds,fumLost,to,penYds,3dConv,4dConv,timePoss
0,201009090nor,"Thursday Sep 9, 2010",Vikings,-5.0,-6.45,-5.54,-0.91,-4.17,-2.86,-4.60,...,15-27-171-1-1,1-9,162,253,1-0,1,6-60,5-13,0-0,26:17
1,201009090nor,"Thursday Sep 9, 2010",Saints,5.0,2.86,4.60,-1.42,0.00,6.45,5.54,...,27-36-237-1-0,1-8,229,308,1-0,0,3-20,3-11,0-0,33:43
0,201009120pit,"Sunday Sep 12, 2010",Falcons,-6.0,-8.90,-4.99,-3.91,-3.92,3.01,0.92,...,27-44-252-0-1,2-15,237,295,0-0,1,3-24,6-16,0-0,30:29
1,201009120pit,"Sunday Sep 12, 2010",Steelers,6.0,-3.01,-0.92,-1.43,-3.79,8.90,4.99,...,18-26-236-0-1,3-25,211,354,0-0,1,4-25,4-14,0-0,32:06
0,201009120oti,"Sunday Sep 12, 2010",Titans,25.0,10.04,6.91,3.79,-4.66,14.46,12.28,...,22-37-180-1-1,4-30,150,286,4-1,2,10-77,3-14,0-2,28:56


In [16]:
# all columns up to firstDowns are fine, so let's look at columns after that
teamGameData[['rushesYdsTds', 'cmpAttYdsTdsInts',
       'sackedYds', 'netPyds', 'totYds', 'fumLost', 'to', 'penYds', '3dConv',
       '4dConv', 'timePoss']].head()

,rushesYdsTds,cmpAttYdsTdsInts,sackedYds,netPyds,totYds,fumLost,to,penYds,3dConv,4dConv,timePoss
0,23-91-0,15-27-171-1-1,1-9,162,253,1-0,1,6-60,5-13,0-0,26:17
1,25-79-1,27-36-237-1-0,1-8,229,308,1-0,0,3-20,3-11,0-0,33:43
0,25-58-0,27-44-252-0-1,2-15,237,295,0-0,1,3-24,6-16,0-0,30:29
1,31-143-1,18-26-236-0-1,3-25,211,354,0-0,1,4-25,4-14,0-0,32:06
0,26-136-0,22-37-180-1-1,4-30,150,286,4-1,2,10-77,3-14,0-2,28:56


In [17]:
# split columns with hyphens into separate columns
teamGameData[['rushAtt','rushYds','rushTds']] = teamGameData['rushesYdsTds'].str.split('-', 2,expand=True)
teamGameData[['passComp','passAtt','passYds','passTds','ints']] = teamGameData['cmpAttYdsTdsInts'].str.split('-', 4,expand=True)
teamGameData[['timesSacked','yardsLostOnSacks']] = teamGameData['sackedYds'].str.split('-', 1,expand=True)
teamGameData[['fumbles','fumblesLost']] = teamGameData['fumLost'].str.split('-', 1,expand=True)
teamGameData[['3dConv','3dAtt']] = teamGameData['3dConv'].str.split('-', 1,expand=True)
teamGameData[['4dConv','4dAtt']] = teamGameData['4dConv'].str.split('-', 1,expand=True)
teamGameData[['3dAtt','3dConv']].head()

,3dAtt,3dConv
0,13,5
1,11,3
0,16,6
1,14,4
0,14,3


In [18]:
# Split timePoss column into minutes/seconds, multiply min column by 60, add the new transformed column to sec
# column for one column that has time of possession in seconds
teamGameData[['min','sec']] = teamGameData['timePoss'].str.split(':', 1,expand=True)
teamGameData[['min','sec']] = teamGameData[['min','sec']].astype('int')
teamGameData['totalSeconds'] = teamGameData['min']*60 + teamGameData['sec']
teamGameData[['totalSeconds']].head()

,totalSeconds
0,1577
1,2023
0,1829
1,1926
0,1736


In [19]:
teamGameData.columns

Index(['gameId', 'date', 'team', 'totExp', 'oTot', 'oPass', 'oRush', 'oTov',
       'dTot', 'dPass', 'dRush', 'dTov', 'stTot', 'stKo', 'stKr', 'stP',
       'stPr', 'fgxp', 'firstDowns', 'rushesYdsTds', 'cmpAttYdsTdsInts',
       'sackedYds', 'netPyds', 'totYds', 'fumLost', 'to', 'penYds', '3dConv',
       '4dConv', 'timePoss', 'rushAtt', 'rushYds', 'rushTds', 'passComp',
       'passAtt', 'passYds', 'passTds', 'ints', 'timesSacked',
       'yardsLostOnSacks', 'fumbles', 'fumblesLost', '3dAtt', '4dAtt', 'min',
       'sec', 'totalSeconds'],
      dtype='object')

In [20]:
# combine 3d and 4d columns
teamGameData[['3dAtt','3dConv']] = teamGameData[['3dAtt','3dConv']].astype('int')
teamGameData[['4dAtt','4dConv']] = teamGameData[['4dAtt','4dConv']].astype('int')
teamGameData['lateDownConv'] = teamGameData['3dConv'] + teamGameData['4dConv']
teamGameData['lateDownAtt'] = teamGameData['3dAtt'] + teamGameData['4dAtt']
teamGameData = teamGameData.drop(columns=['3dConv','4dConv','3dAtt','4dAtt','rushesYdsTds', 'cmpAttYdsTdsInts',
                                          'sackedYds', 'fumLost', 'penYds', 'timePoss', 'min', 'sec'])
teamGameData.columns

Index(['gameId', 'date', 'team', 'totExp', 'oTot', 'oPass', 'oRush', 'oTov',
       'dTot', 'dPass', 'dRush', 'dTov', 'stTot', 'stKo', 'stKr', 'stP',
       'stPr', 'fgxp', 'firstDowns', 'netPyds', 'totYds', 'to', 'rushAtt',
       'rushYds', 'rushTds', 'passComp', 'passAtt', 'passYds', 'passTds',
       'ints', 'timesSacked', 'yardsLostOnSacks', 'fumbles', 'fumblesLost',
       'totalSeconds', 'lateDownConv', 'lateDownAtt'],
      dtype='object')

In [21]:
teamGameData.head()

,gameId,date,team,totExp,oTot,oPass,oRush,oTov,dTot,dPass,...,passYds,passTds,ints,timesSacked,yardsLostOnSacks,fumbles,fumblesLost,totalSeconds,lateDownConv,lateDownAtt
0,201009090nor,"Thursday Sep 9, 2010",Vikings,-5.0,-6.45,-5.54,-0.91,-4.17,-2.86,-4.60,...,171,1,1,1,9,1,0,1577,5,13
1,201009090nor,"Thursday Sep 9, 2010",Saints,5.0,2.86,4.60,-1.42,0.00,6.45,5.54,...,237,1,0,1,8,1,0,2023,3,11
0,201009120pit,"Sunday Sep 12, 2010",Falcons,-6.0,-8.90,-4.99,-3.91,-3.92,3.01,0.92,...,252,0,1,2,15,0,0,1829,6,16
1,201009120pit,"Sunday Sep 12, 2010",Steelers,6.0,-3.01,-0.92,-1.43,-3.79,8.90,4.99,...,236,0,1,3,25,0,0,1926,4,14
0,201009120oti,"Sunday Sep 12, 2010",Titans,25.0,10.04,6.91,3.79,-4.66,14.46,12.28,...,180,1,1,4,30,4,1,1736,3,16


In [22]:
finalTeamGameDataTable = teamGameData

In [23]:
%store finalTeamGameDataTable

Stored 'finalTeamGameDataTable' (DataFrame)


In [24]:
finalTeamGameDataTable.head()

,gameId,date,team,totExp,oTot,oPass,oRush,oTov,dTot,dPass,...,passYds,passTds,ints,timesSacked,yardsLostOnSacks,fumbles,fumblesLost,totalSeconds,lateDownConv,lateDownAtt
0,201009090nor,"Thursday Sep 9, 2010",Vikings,-5.0,-6.45,-5.54,-0.91,-4.17,-2.86,-4.60,...,171,1,1,1,9,1,0,1577,5,13
1,201009090nor,"Thursday Sep 9, 2010",Saints,5.0,2.86,4.60,-1.42,0.00,6.45,5.54,...,237,1,0,1,8,1,0,2023,3,11
0,201009120pit,"Sunday Sep 12, 2010",Falcons,-6.0,-8.90,-4.99,-3.91,-3.92,3.01,0.92,...,252,0,1,2,15,0,0,1829,6,16
1,201009120pit,"Sunday Sep 12, 2010",Steelers,6.0,-3.01,-0.92,-1.43,-3.79,8.90,4.99,...,236,0,1,3,25,0,0,1926,4,14
0,201009120oti,"Sunday Sep 12, 2010",Titans,25.0,10.04,6.91,3.79,-4.66,14.46,12.28,...,180,1,1,4,30,4,1,1736,3,16


In [25]:
# Turn Pro football reference data into rolling averages. Name columns accordingly

finalTeamGameDataTable['date'] = pd.to_datetime(finalTeamGameDataTable['date'])

rollAvgGameData = finalTeamGameDataTable.set_index(['date', 'team'])
rollAvgGameData

varDict = {'totExp':'totExpAvg','oTot':'oTotAvg','oPass':'oPassAvg','oRush':'oRushAvg','oTov':'oTovAvg','dTot':'dTotAvg','dPass':'dPassAvg',
           'dRush':'dRushAvg','dTov':'dTovAvg', 'stTot':'stTotAvg', 'stKo':'stKoAvg', 'stKr':'stKrAvg', 'stP':'stPAvg',
           'stPr':'stPrAvg', 'fgxp':'fgxpAvg', 'firstDowns':'firstDownsAvg', 'netPyds':'netPydsAvg', 'totYds':'totYdsAvg', 'to':'toAvg', 'rushAtt':'rushAttAvg',
           'rushYds':'rushYdsAvg', 'rushTds':'rushTdsAvg', 'passComp':'passCompAvg', 'passAtt':'passAttAvg', 'passYds':'passYdsAvg', 'passTds':'passTdsAvg',
           'ints':'intsAvg', 'timesSacked':'timesSackedAvg', 'yardsLostOnSacks':'yardsLostOnSacksAvg','totalSeconds':'totalSecondsAvg', 'lateDownConv':'lateDownConvAvg', 'lateDownAtt':'lateDownAttAvg'}

for key, val in varDict.items():
    rollAvgGameData[val] = rollAvgGameData.groupby('team')[key].apply(lambda x: x.shift(1).rolling(window=7).mean())
    
rollAvgGameData = rollAvgGameData.fillna(method='bfill')
rollAvgGameData = rollAvgGameData.reset_index()

rollAvgGameData[['date','team','totExp','totExpAvg']].sort_values(by=['team','date'])
rollAvgGameData.columns

Index(['date', 'team', 'gameId', 'totExp', 'oTot', 'oPass', 'oRush', 'oTov',
       'dTot', 'dPass', 'dRush', 'dTov', 'stTot', 'stKo', 'stKr', 'stP',
       'stPr', 'fgxp', 'firstDowns', 'netPyds', 'totYds', 'to', 'rushAtt',
       'rushYds', 'rushTds', 'passComp', 'passAtt', 'passYds', 'passTds',
       'ints', 'timesSacked', 'yardsLostOnSacks', 'fumbles', 'fumblesLost',
       'totalSeconds', 'lateDownConv', 'lateDownAtt', 'totExpAvg', 'oTotAvg',
       'oPassAvg', 'oRushAvg', 'oTovAvg', 'dTotAvg', 'dPassAvg', 'dRushAvg',
       'dTovAvg', 'stTotAvg', 'stKoAvg', 'stKrAvg', 'stPAvg', 'stPrAvg',
       'fgxpAvg', 'firstDownsAvg', 'netPydsAvg', 'totYdsAvg', 'toAvg',
       'rushAttAvg', 'rushYdsAvg', 'rushTdsAvg', 'passCompAvg', 'passAttAvg',
       'passYdsAvg', 'passTdsAvg', 'intsAvg', 'timesSackedAvg',
       'yardsLostOnSacksAvg', 'totalSecondsAvg', 'lateDownConvAvg',
       'lateDownAttAvg'],
      dtype='object')

In [26]:
rollAvgGameData.to_csv('C://Users//Michael//Desktop//Springboard//Capstone2//csvFiles//rollAvgGameData.csv')

In [30]:
# Now let's setup our home/away matchup columns with rolling averages
gameTable = pd.read_csv('C://Users//Michael//Desktop//Springboard//Capstone1//dataCleaning//cleanedData//cleanedGameData.csv')
dvoaTable = pd.read_csv('C://Users//Michael//Desktop//Springboard//Capstone1//dataCleaning//cleanedData//cleanedDvoaData.csv', encoding='windows-1252')
pffTable = pd.read_csv('C://Users//Michael//Desktop//Springboard//Capstone1//dataCleaning//cleanedData//cleanedPffData.csv')

In [47]:
pffTable['date'] = pd.to_datetime(pffTable['date'])
rollAvgPff = pffTable.set_index('date')
rollAvgPff

,season,week,team,opp,time,w,pf,pa,overall,off,...,run,rblk,def,rdef,tack,prsh,cov,spec,lateGame,year
date,,,,,,,,,,,,,,,,,,,,,
2006-09-10,2006,1,Arizona Cardinals,San Francisco 49ers,11:00am,W,34,27,72.5,72.6,...,47.4,65.6,67.7,69.8,37.5,56.2,70.7,-,False,2006
2006-09-17,2006,2,Arizona Cardinals,Seattle Seahawks,11:00am,L,10,21,48.0,50.1,...,59.7,62.1,49.8,38.3,44.7,60.5,62.3,-,False,2006
2006-09-24,2006,3,Arizona Cardinals,Los Angeles Rams,11:00am,L,14,16,65.6,66.0,...,54.1,62.8,62.9,80.1,57.1,51.7,53.8,-,False,2006
2006-10-01,2006,4,Arizona Cardinals,Atlanta Falcons,8:00am,L,10,32,64.1,55.5,...,41.9,61.9,73.0,68.1,82.9,56.8,75.3,-,False,2006
2006-10-08,2006,5,Arizona Cardinals,Kansas City Chiefs,8:00am,L,20,23,65.6,63.4,...,57.3,65.3,66.4,78.9,53.0,63.5,56.9,-,False,2006
2006-10-16,2006,6,Arizona Cardinals,Chicago Bears,12:00am,L,23,24,66.0,55.4,...,34.4,62.1,81.6,65.9,64.6,73.7,82.3,-,False,2006
2006-10-22,2006,7,Arizona Cardinals,Oakland Raiders,9:00am,L,9,22,61.5,54.6,...,38.8,55.5,68.1,74.4,89.2,56.0,63.2,-,False,2006
2006-10-29,2006,8,Arizona Cardinals,Green Bay Packers,7:00am,L,14,31,55.0,60.3,...,47.6,70.6,49.5,56.0,73.1,52.5,45.4,-,False,2006
2006-11-12,2006,10,Arizona Cardinals,Dallas Cowboys,7:00am,L,10,27,51.0,54.1,...,53.3,51.6,50.5,48.1,33.0,57.8,51.8,-,False,2006


In [34]:
# First, turn date cols into dt objects
gameTable['date'] = pd.to_datetime(gameTable['date'])
pffTable['date'] = pd.to_datetime(pffTable['date'])

# Create a table of running averages for pff grades and game scores. Use previous 7-game averages for a team's variables
rollAvgPff = pffTable.set_index('date')
varDict = {'pf':'avgPf','pa':'avgPa','overall':'avgOvr','off':'avgOff','pass':'avgPass','pblk':'avgPblk','recv':'avgRecv',
           'run':'avgRun','rblk':'avgRblk','def':'avgDef','rdef':'avgRdef','tack':'avgTack','prsh':'avgPrsh','cov':'avgCov'}

for key, val in varDict.items():
    rollAvgPff[val] = rollAvgPff.groupby('team')[key].apply(lambda x: x.shift(1).rolling(window=7).mean())

rollAvgPff = rollAvgPff.fillna(method='bfill')
rollAvgPff = rollAvgPff.reset_index()

# Need to make new rollingAvgPff tables that have all dates covered... Dates in pff table are off by one or two days
# Strategy is to create new df's that have all dates, then join the normal pff table to each of them by date,
# then merge all the tables together. All dates will be in the final table
pffB1 = rollAvgPff.copy()
pffB1['date'] = pffB1['date'] - datetime.timedelta(days=1)
pffB2 = rollAvgPff.copy()
pffB2['date'] = pffB2['date'] - datetime.timedelta(days=2)
pffF1 = rollAvgPff.copy()
pffF1['date'] = pffF1['date'] + datetime.timedelta(days=1)
pffF2 = rollAvgPff.copy()
pffF2['date'] = pffF2['date'] + datetime.timedelta(days=2)

# Now join pffTable to each new df
t0 = pd.merge(gameTable,rollAvgPff,how='inner',left_on=['date','awayTeam'], right_on=['date','team'])
t1 = pd.merge(gameTable,pffB1,how='inner',left_on=['date','awayTeam'], right_on=['date','team'])
t2 = pd.merge(gameTable,pffB2,how='inner',left_on=['date','awayTeam'], right_on=['date','team'])
t3 = pd.merge(gameTable,pffF1,how='inner',left_on=['date','awayTeam'], right_on=['date','team'])
t4 = pd.merge(gameTable,pffF2,how='inner',left_on=['date','awayTeam'], right_on=['date','team'])
t5 = pd.merge(gameTable,rollAvgPff,how='inner',left_on=['date','homeTeam'], right_on=['date','team'])
t6 = pd.merge(gameTable,pffB1,how='inner',left_on=['date','homeTeam'], right_on=['date','team'])
t7 = pd.merge(gameTable,pffB2,how='inner',left_on=['date','homeTeam'], right_on=['date','team'])
t8 = pd.merge(gameTable,pffF1,how='inner',left_on=['date','homeTeam'], right_on=['date','team'])
t9 = pd.merge(gameTable,pffF2,how='inner',left_on=['date','homeTeam'], right_on=['date','team'])

awayTable = t0.append([t1,t2,t3,t4],ignore_index=True)
homeTable = t5.append([t6,t7,t8,t9],ignore_index=True)

awayTable['homeAway'] = 'away'
homeTable['homeAway'] = 'home'

awayTable = awayTable[['gameId', 'date','season', 'week','team','homeAway',
       'stadium', 'roof', 'surface', 'avgPf', 'avgPa', 'avgOvr', 'avgOff',
       'avgPass', 'avgPblk', 'avgRecv', 'avgRun', 'avgRblk', 'avgDef', 'avgRdef',
       'avgPrsh', 'avgCov','pf','pa','totalScore', 'overUnder', 'result']]
homeTable = homeTable[['gameId', 'date','season', 'week','team','homeAway',
       'stadium', 'roof', 'surface', 'avgPf', 'avgPa', 'avgOvr', 'avgOff',
       'avgPass', 'avgPblk', 'avgRecv', 'avgRun', 'avgRblk', 'avgDef', 'avgRdef',
       'avgPrsh', 'avgCov','pf','pa','totalScore', 'overUnder', 'result']]

# Transform week from dvoa table into correct format for analysis (add one)
dvoaTable['week'] = dvoaTable['week'] + 1

# Next, join these tables with DVOA table
awayTable = awayTable.merge(dvoaTable, on=['season','week','team'])
homeTable = homeTable.merge(dvoaTable, on=['season','week','team'])

# We only want the team's offensive and total grade columns and the team's opponent's
# defensive/total grade cols
# We can make two copies of both tables: awayOff, awayDef, homeOff, homeDef. Then join
# awayOff with homeDef, and join homeOff with awayDef and predict team totals for each game
# after transforming variables by combining columns.
awayOff = awayTable[['gameId', 'date', 'season', 'week', 'team', 'homeAway', 'stadium',
       'roof', 'surface', 'avgPf', 'avgOvr', 'avgOff', 'avgPass',
       'avgPblk', 'avgRecv', 'avgRun', 'avgRblk', 'totalDvoa', 'offDvoa', 'stDvoa',
       'overUnder', 'pf', 'pa', 'totalScore']]
awayDef = awayTable[['gameId', 'date', 'season', 'week', 'team', 'stadium',
       'roof', 'surface', 'avgPa', 'avgOvr', 'avgDef', 'avgRdef',
       'avgPrsh', 'avgCov', 'totalDvoa', 'defDvoa', 'totalScore']]
homeOff = homeTable[['gameId', 'date', 'season', 'week', 'team', 'homeAway', 'stadium',
       'roof', 'surface', 'avgPf', 'avgOvr', 'avgOff', 'avgPass',
       'avgPblk', 'avgRecv', 'avgRun', 'avgRblk', 'totalDvoa', 'offDvoa', 'stDvoa',
       'overUnder', 'pf', 'pa', 'totalScore']]
homeDef = homeTable[['gameId', 'date', 'season', 'week', 'team', 'stadium',
       'roof', 'surface', 'avgPa', 'avgOvr', 'avgDef', 'avgRdef',
       'avgPrsh', 'avgCov', 'totalDvoa', 'defDvoa', 'totalScore']]

awayMatchup = awayOff.merge(homeDef, on=['gameId', 'date', 'season', 'week', 'stadium', 'roof', 'surface', 'totalScore'])
homeMatchup = homeOff.merge(awayDef, on=['gameId', 'date', 'season', 'week', 'stadium', 'roof', 'surface', 'totalScore'])

# Now we will create new columns for these two tables by combining matchups
# Ex) add awayMatchup['avgOff'] + awayMatchup['avgDef'] ('avgDef' is the away team's opponent's average defensive grade)
# to turn these two columns into one

awayMatchup['ptsMatchup'] = awayMatchup['avgPf'] + awayMatchup['avgPa']
awayMatchup['offMatchup'] = awayMatchup['avgOff'] - awayMatchup['avgDef']
awayMatchup['ovrMatchup'] = awayMatchup['avgOvr_x'] - awayMatchup['avgOvr_y']
awayMatchup['passMatchup'] = awayMatchup['avgPass'] - awayMatchup['avgCov']
awayMatchup['pblkMatchup'] = awayMatchup['avgPblk'] - awayMatchup['avgPrsh']
awayMatchup['runMatchup'] = awayMatchup['avgRun'] - awayMatchup['avgRdef']
awayMatchup['offDvoaMatchup'] = awayMatchup['offDvoa'] + awayMatchup['defDvoa']
awayMatchup['totalDvoaMatchup'] = awayMatchup['totalDvoa_x'] - awayMatchup['totalDvoa_y']

homeMatchup['ptsMatchup'] = homeMatchup['avgPf'] + homeMatchup['avgPa']
homeMatchup['offMatchup'] = homeMatchup['avgOff'] - homeMatchup['avgDef']
homeMatchup['ovrMatchup'] = homeMatchup['avgOvr_x'] - homeMatchup['avgOvr_y']
homeMatchup['passMatchup'] = homeMatchup['avgPass'] - homeMatchup['avgCov']
homeMatchup['pblkMatchup'] = homeMatchup['avgPblk'] - homeMatchup['avgPrsh']
homeMatchup['runMatchup'] = homeMatchup['avgRun'] - homeMatchup['avgRdef']
homeMatchup['offDvoaMatchup'] = homeMatchup['offDvoa'] + homeMatchup['defDvoa']
homeMatchup['totalDvoaMatchup'] = homeMatchup['totalDvoa_x'] - homeMatchup['totalDvoa_y']

cols = list(awayMatchup.columns)
cols
cols.pop(21)
cols.append('pf')

awayMatchup = awayMatchup[cols]
homeMatchup = homeMatchup[cols]

In [35]:
awayMatchup.columns

Index(['gameId', 'date', 'season', 'week', 'team_x', 'homeAway', 'stadium',
       'roof', 'surface', 'avgPf', 'avgOvr_x', 'avgOff', 'avgPass', 'avgPblk',
       'avgRecv', 'avgRun', 'avgRblk', 'totalDvoa_x', 'offDvoa', 'stDvoa',
       'overUnder', 'pa', 'totalScore', 'team_y', 'avgPa', 'avgOvr_y',
       'avgDef', 'avgRdef', 'avgPrsh', 'avgCov', 'totalDvoa_y', 'defDvoa',
       'ptsMatchup', 'offMatchup', 'ovrMatchup', 'passMatchup', 'pblkMatchup',
       'runMatchup', 'offDvoaMatchup', 'totalDvoaMatchup', 'pf'],
      dtype='object')

In [40]:
dvoaTable.head()

,season,week,team,totalDvoa,offDvoa,offRank,defDvoa,defRank,stDvoa,stRank
0,2007,2,New England Patriots,0.319,0.257,1,-0.051,9,0.011,7
1,2007,2,Philadelphia Eagles,0.260,0.222,3,-0.039,12,-0.001,18
2,2007,2,Jacksonville Jaguars,0.247,0.082,6,-0.166,1,-0.001,19
3,2007,2,Baltimore Ravens,0.120,0.060,11,-0.065,7,-0.005,22
4,2007,2,Washington Redskins,0.112,-0.018,15,-0.115,4,0.015,3


In [36]:
awayMatchup.to_csv('C://Users//Michael//Desktop//Springboard//Capstone2//csvFiles//awayMatchup.csv')
homeMatchup.to_csv('C://Users//Michael//Desktop//Springboard//Capstone2//csvFiles//homeMatchup.csv')